# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [516]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 289553 # по умолчанию вписан мой номер

2021-12-21 13:27:57.150697


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1 (6 баллов)

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [517]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Аптеки', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Алименты', 'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Спиртное и алкоголь', 'Предсказатели будущего', 'Азартные игры']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else: 
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Азартные игры', 'Спиртное и алкоголь', 'Отели', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Алименты', 'Магазины косметики и Салоны красоты', 'Аптеки']
В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 14 признаков, то есть семь категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [518]:
%%capture
import pandas as pd
!wget https://www.dropbox.com/s/axkihywa8q118rk/transactions.csv
!wget https://www.dropbox.com/s/xwbkjwik600clsh/tr_mcc_codes.csv
!wget https://www.dropbox.com/s/5deipwkmeial1ba/tr_types.csv
!wget https://www.dropbox.com/s/qc307hdkc9jdxf8/gender_train.csv

transactions = pd.read_csv('transactions.csv')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';')
gender_train = pd.read_csv('gender_train.csv')
tr_types = pd.read_csv('tr_types.csv',sep=';')

In [519]:
display(transactions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6849346 entries, 0 to 6849345
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   customer_id  int64  
 1   tr_datetime  object 
 2   mcc_code     int64  
 3   tr_type      int64  
 4   amount       float64
 5   term_id      object 
dtypes: float64(1), int64(3), object(2)
memory usage: 313.5+ MB


None

### **Для каждой категории выведите соответствующие ей mcc-коды с описаниями.**

In [520]:
display(tr_mcc_codes.head())

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело


In [521]:
tr_code_azart_games = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("азартным"))]
display(tr_code_azart_games)

,mcc_code,mcc_description
162,7995,Транзакции по азартным играм


In [522]:
tr_code_alcohol = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("спирт"))
| ( tr_mcc_codes['mcc_description'].str.contains("алко"))]
display(tr_code_alcohol)

,mcc_code,mcc_description
91,5813,"Бары, коктейль-бары, дискотеки, ночные клубы и..."
95,5921,Магазины с продажей спиртных напитков на вынос...


In [523]:
tr_code_hotels = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("Отели"))] 
display(tr_code_hotels)

,mcc_code,mcc_description
131,7011,"Отели, мотели, базы отдыха, сервисы бронирования"


In [524]:
tr_code_car = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("Автозапчасти"))
| ( tr_mcc_codes['mcc_description'].str.contains("СТО")) | ( tr_mcc_codes['mcc_description'].str.contains("Автошины")) ]
display(tr_code_car)

,mcc_code,mcc_description
63,5532,Автошины
64,5533,Автозапчасти и аксессуары
149,7538,СТО общего назначения


In [525]:
tr_code_aliments = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("алименты"))] 
display(tr_code_aliments)

,mcc_code,mcc_description
179,9211,"Судовые выплаты, включая алименты и детскую по..."


In [526]:
tr_code_cosmetics = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("красоты"))
| ( tr_mcc_codes['mcc_description'].str.contains("косметики"))] 
display(tr_code_cosmetics)

,mcc_code,mcc_description
116,5977,Магазины косметики
135,7230,Салоты красоты и парикмахерские


In [527]:
tr_code_apteka = tr_mcc_codes[['mcc_code', 'mcc_description']][( tr_mcc_codes['mcc_description'].str.contains("Аптеки"))]
display(tr_code_apteka)

,mcc_code,mcc_description
94,5912,Аптеки


In [528]:
data1 = transactions[(transactions['mcc_code'].isin(tr_code_azart_games['mcc_code']))]
data1_pos = data1[data1['amount']> 0].groupby(['customer_id'])['amount'].sum()
data1_neg = data1[data1['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [529]:
data2 = transactions[(transactions['mcc_code'].isin(tr_code_alcohol['mcc_code']))]
data2_pos = data2[data2['amount']> 0].groupby(['customer_id'])['amount'].sum()
data2_neg = data2[data2['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [530]:
data3 = transactions[(transactions['mcc_code'].isin(tr_code_hotels['mcc_code']))]
data3_pos = data3[data3['amount']> 0].groupby(['customer_id'])['amount'].sum()
data3_neg = data3[data3['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [531]:
data4 = transactions[(transactions['mcc_code'].isin(tr_code_car['mcc_code']))]
data4_pos = data4[data4['amount']> 0].groupby(['customer_id'])['amount'].sum()
data4_neg = data4[data4['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [532]:
data5 = transactions[(transactions['mcc_code'].isin(tr_code_aliments['mcc_code']))]
data5_pos = data5[data5['amount']> 0].groupby(['customer_id'])['amount'].sum()
data5_neg = data5[data5['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [533]:
data6 = transactions[(transactions['mcc_code'].isin(tr_code_cosmetics['mcc_code']))]
data6_pos = data6[data6['amount']> 0].groupby(['customer_id'])['amount'].sum()
data6_neg = data6[data6['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [534]:
data7 = transactions[(transactions['mcc_code'].isin(tr_code_apteka['mcc_code']))]
data7_pos = data7[data7['amount']> 0].groupby(['customer_id'])['amount'].sum()
data7_neg = data7[data7['amount']< 0].groupby(['customer_id'])['amount'].sum()

In [535]:
data1_neg.name = 'azart_games_spend'
data1_pos.name = 'azart_games_earn'
data2_neg.name = 'alcohol_spend'
data2_pos.name = 'alcohol_earn'
data3_neg.name = 'hotels_spend'
data3_pos.name = 'hotels_earn'
data4_neg.name = 'car_spend'
data4_pos.name = 'car_earn'
data5_neg.name = 'aliments_spend'
data5_pos.name = 'aliments_earn'
data6_neg.name = 'cosmetics_spend'
data6_pos.name = 'cosmetics_earn'
data7_neg.name = 'apteka_spend'
data7_pos.name = 'apteka_earn'
column_names = ['azart_games_spend', "azart_games_earn", 'alcohol_spend', 'alcohol_earn', 'hotels_spend', 'hotels_earn', 'car_spend',
                'car_earn','aliments_spend','aliments_earn','cosmetics_spend','cosmetics_earn', 'apteka_spend','apteka_earn' ]

In [536]:
data_1 = pd.concat([data1_pos, data1_neg], axis=1)
data_2 = pd.concat([data2_pos, data2_neg], axis=1)
data_3 = pd.concat([data3_pos, data3_neg], axis=1)
data_4 = pd.concat([data4_pos, data4_neg], axis=1)
data_5 = pd.concat([data5_pos, data5_neg], axis=1)
data_6 = pd.concat([data6_pos, data6_neg], axis=1)
data_7 = pd.concat([data7_pos, data7_neg], axis=1)
data = pd.merge(data_1,data_2, on='customer_id', how='outer')
data = pd.merge(data,data_3, on='customer_id', how='outer')
data = pd.merge(data,data_4, on='customer_id', how='outer')
data = pd.merge(data,data_5, on='customer_id', how='outer')
data = pd.merge(data,data_6, on='customer_id', how='outer')
data = pd.merge(data,data_7, on='customer_id', how='outer')

In [537]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11932 entries, 302489 to 99984336
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   azart_games_earn   59 non-null     float64
 1   azart_games_spend  523 non-null    float64
 2   alcohol_earn       18 non-null     float64
 3   alcohol_spend      5721 non-null   float64
 4   hotels_earn        28 non-null     float64
 5   hotels_spend       1308 non-null   float64
 6   car_earn           129 non-null    float64
 7   car_spend          4188 non-null   float64
 8   aliments_earn      0 non-null      float64
 9   aliments_spend     11 non-null     float64
 10  cosmetics_earn     23 non-null     float64
 11  cosmetics_spend    6252 non-null   float64
 12  apteka_earn        19 non-null     float64
 13  apteka_spend       10668 non-null  float64
dtypes: float64(14)
memory usage: 1.4 MB


None

In [538]:
data = pd.merge(data,gender_train,how='outer', on='customer_id')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13640 entries, 0 to 13639
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        13640 non-null  int64  
 1   azart_games_earn   59 non-null     float64
 2   azart_games_spend  523 non-null    float64
 3   alcohol_earn       18 non-null     float64
 4   alcohol_spend      5721 non-null   float64
 5   hotels_earn        28 non-null     float64
 6   hotels_spend       1308 non-null   float64
 7   car_earn           129 non-null    float64
 8   car_spend          4188 non-null   float64
 9   aliments_earn      0 non-null      float64
 10  aliments_spend     11 non-null     float64
 11  cosmetics_earn     23 non-null     float64
 12  cosmetics_spend    6252 non-null   float64
 13  apteka_earn        19 non-null     float64
 14  apteka_spend       10668 non-null  float64
 15  gender             8400 non-null   float64
dtypes: float64(15), int64(

None

### **По какому признаку самое большое количество непустых значений?**

Из информации о данных видно что траты в аптеке содержат самое большое количество непустых значений - 10668

### **По какой категории самая большая разница между мужчинами и женщинами?**

In [539]:
import numpy as np
gender_0_transactions = data[data['gender']==0.0]
gender_1_transactions = data[data['gender']==1.0]
gender_0_column_sums = {}
gender_1_column_sums = {}
diff = {}
for column in column_names:
  gender_0_column_sums[column] = gender_0_transactions[column].sum()
  gender_1_column_sums[column] = gender_1_transactions[column].sum()

for column in column_names:
  diff[column] = (np.abs(gender_0_column_sums[column] - gender_1_column_sums[column])) 

print(diff)
print(max(diff, key=diff.get))

{'azart_games_spend': 310211834.1499993, 'azart_games_earn': 68037756.95999998, 'alcohol_spend': 125028610.87999997, 'alcohol_earn': 395369.67, 'hotels_spend': 280252668.26, 'hotels_earn': 435112.49, 'car_spend': 468920776.9499999, 'car_earn': 2503579.79, 'aliments_spend': 822005.23, 'aliments_earn': 0.0, 'cosmetics_spend': 356563525.0400001, 'cosmetics_earn': 719494.8400000001, 'apteka_spend': 236958262.21000004, 'apteka_earn': 133418.16999999998}
car_spend


**По тратам на машины**

## Вопросы к заданию 1

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.
2. По какому признакому самое большое количество непустых значений?
3. По какой категории самая большая разница между мужчинами и женщинами?

# Задание 2 (3 балла)

### Обработка данных

In [540]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13640 entries, 0 to 13639
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        13640 non-null  int64  
 1   azart_games_earn   59 non-null     float64
 2   azart_games_spend  523 non-null    float64
 3   alcohol_earn       18 non-null     float64
 4   alcohol_spend      5721 non-null   float64
 5   hotels_earn        28 non-null     float64
 6   hotels_spend       1308 non-null   float64
 7   car_earn           129 non-null    float64
 8   car_spend          4188 non-null   float64
 9   aliments_earn      0 non-null      float64
 10  aliments_spend     11 non-null     float64
 11  cosmetics_earn     23 non-null     float64
 12  cosmetics_spend    6252 non-null   float64
 13  apteka_earn        19 non-null     float64
 14  apteka_spend       10668 non-null  float64
 15  gender             8400 non-null   float64
dtypes: float64(15), int64(

None

In [541]:
# удалим транзакции гендер которых нам не известен так как у нас обучение с учителем 
data = data.dropna(subset=['gender'])
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8400 entries, 1 to 13639
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        8400 non-null   int64  
 1   azart_games_earn   33 non-null     float64
 2   azart_games_spend  301 non-null    float64
 3   alcohol_earn       14 non-null     float64
 4   alcohol_spend      3196 non-null   float64
 5   hotels_earn        14 non-null     float64
 6   hotels_spend       732 non-null    float64
 7   car_earn           76 non-null     float64
 8   car_spend          2349 non-null   float64
 9   aliments_earn      0 non-null      float64
 10  aliments_spend     5 non-null      float64
 11  cosmetics_earn     16 non-null     float64
 12  cosmetics_spend    3539 non-null   float64
 13  apteka_earn        11 non-null     float64
 14  apteka_spend       5995 non-null   float64
 15  gender             8400 non-null   float64
dtypes: float64(15), int64(1

None

In [542]:
#  удалим колнку customer_id она уже не нужна
data = data.drop('customer_id', axis =1)
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8400 entries, 1 to 13639
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   azart_games_earn   33 non-null     float64
 1   azart_games_spend  301 non-null    float64
 2   alcohol_earn       14 non-null     float64
 3   alcohol_spend      3196 non-null   float64
 4   hotels_earn        14 non-null     float64
 5   hotels_spend       732 non-null    float64
 6   car_earn           76 non-null     float64
 7   car_spend          2349 non-null   float64
 8   aliments_earn      0 non-null      float64
 9   aliments_spend     5 non-null      float64
 10  cosmetics_earn     16 non-null     float64
 11  cosmetics_spend    3539 non-null   float64
 12  apteka_earn        11 non-null     float64
 13  apteka_spend       5995 non-null   float64
 14  gender             8400 non-null   float64
dtypes: float64(15)
memory usage: 1.0 MB


None

In [543]:
# нормируем данные от 0 до 1
target = data['gender'].values
normalized_df=(data-data.mean())/data.std()
display(normalized_df.head())

,azart_games_earn,azart_games_spend,alcohol_earn,alcohol_spend,hotels_earn,hotels_spend,car_earn,car_spend,aliments_earn,aliments_spend,cosmetics_earn,cosmetics_spend,apteka_earn,apteka_spend,gender
1,NaN,0.236421,NaN,NaN,NaN,NaN,NaN,0.136713,NaN,NaN,NaN,-0.109956,NaN,-0.183465,-0.889998
3,NaN,0.236502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356475,1.123464
6,NaN,0.233922,NaN,-0.059270,NaN,0.265465,NaN,NaN,NaN,NaN,NaN,0.245660,NaN,0.326312,1.123464
9,NaN,0.231820,NaN,0.169178,NaN,NaN,NaN,-0.062521,NaN,NaN,NaN,0.051817,NaN,-0.327369,1.123464
13,NaN,0.205020,NaN,0.117372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.123464


In [544]:
# заменим nan на 0 и поменяет знак  трат на положительный 
data = normalized_df.fillna(0) 
data = data.abs()
display(data.head())

,azart_games_earn,azart_games_spend,alcohol_earn,alcohol_spend,hotels_earn,hotels_spend,car_earn,car_spend,aliments_earn,aliments_spend,cosmetics_earn,cosmetics_spend,apteka_earn,apteka_spend,gender
1,0.0,0.236421,0.0,0.000000,0.0,0.000000,0.0,0.136713,0.0,0.0,0.0,0.109956,0.0,0.183465,0.889998
3,0.0,0.236502,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.356475,1.123464
6,0.0,0.233922,0.0,0.059270,0.0,0.265465,0.0,0.000000,0.0,0.0,0.0,0.245660,0.0,0.326312,1.123464
9,0.0,0.231820,0.0,0.169178,0.0,0.000000,0.0,0.062521,0.0,0.0,0.0,0.051817,0.0,0.327369,1.123464
13,0.0,0.205020,0.0,0.117372,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,1.123464


In [545]:
print(target)
data = data.drop('gender', axis=1)
display(data.head())

[0. 1. 1. ... 1. 0. 0.]


,azart_games_earn,azart_games_spend,alcohol_earn,alcohol_spend,hotels_earn,hotels_spend,car_earn,car_spend,aliments_earn,aliments_spend,cosmetics_earn,cosmetics_spend,apteka_earn,apteka_spend
1,0.0,0.236421,0.0,0.000000,0.0,0.000000,0.0,0.136713,0.0,0.0,0.0,0.109956,0.0,0.183465
3,0.0,0.236502,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.356475
6,0.0,0.233922,0.0,0.059270,0.0,0.265465,0.0,0.000000,0.0,0.0,0.0,0.245660,0.0,0.326312
9,0.0,0.231820,0.0,0.169178,0.0,0.000000,0.0,0.062521,0.0,0.0,0.0,0.051817,0.0,0.327369
13,0.0,0.205020,0.0,0.117372,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000


In [546]:
from sklearn.model_selection import train_test_split
X_train, X_test ,y_train, y_test = train_test_split(data, target, test_size = 0.3) 

### **Mlp model**

In [579]:
from keras import Sequential
import tensorflow as tf
from keras.layers import Dense, Dropout

Модель - многослойная нейронная сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Настроить параметры модели.

In [615]:
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(2042, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(2, activation='softmax'))

In [616]:
model.compile(loss="sparse_categorical_crossentropy",
                   optimizer='nadam',
                   metrics=['accuracy'])

In [617]:
history = model.fit(X_train, y_train, epochs=12, validation_data=(X_test, y_test))

Epoch 1/12
184/184 [==============================] - 16s 80ms/step - loss: 0.6519 - accuracy: 0.6442 - val_loss: 0.6568 - val_accuracy: 0.6460
Epoch 2/12
184/184 [==============================] - 15s 79ms/step - loss: 0.6322 - accuracy: 0.6534 - val_loss: 0.6234 - val_accuracy: 0.6619
Epoch 3/12
184/184 [==============================] - 19s 102ms/step - loss: 0.6220 - accuracy: 0.6585 - val_loss: 0.6182 - val_accuracy: 0.6683
Epoch 4/12
184/184 [==============================] - 14s 77ms/step - loss: 0.6162 - accuracy: 0.6655 - val_loss: 0.6200 - val_accuracy: 0.6623
Epoch 5/12
184/184 [==============================] - 18s 96ms/step - loss: 0.6131 - accuracy: 0.6672 - val_loss: 0.6150 - val_accuracy: 0.6623
Epoch 6/12
184/184 [==============================] - 15s 82ms/step - loss: 0.6058 - accuracy: 0.6662 - val_loss: 0.6167 - val_accuracy: 0.6690
Epoch 7/12
184/184 [==============================] - 15s 81ms/step - loss: 0.6066 - accuracy: 0.6738 - val_loss: 0.6110 - val_accuracy

## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены примерные, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация на скрытых слоях: ReLU
* и т.д.

2. Сколько у вашей модели параметров?

In [619]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_100 (Dense)           (None, 256)               3840      
                                                                 
 dense_101 (Dense)           (None, 1024)              263168    
                                                                 
 dense_102 (Dense)           (None, 2042)              2093050   
                                                                 
 dense_103 (Dense)           (None, 512)               1046016   
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_104 (Dense)           (None, 2)                 1026      
                                                                 
Total params: 3,407,100
Trainable params: 3,407,100
N

## Ответы:

* количество эпох: 12
* скорость обучения: используется оптимизатор nadam у него default lr=0.001
* функция активация на скрытых слоях: ReLU


# Задание 3 (2 балла)


Проверить качество модели не менее, чем на трёх разных метриках. 

## Вопросы к заданию 3

1. Выведите значения метрик.
2. Как улучшить модель?
3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

In [618]:
print(history.history['val_accuracy'])

[0.6460317373275757, 0.6619047522544861, 0.6682539582252502, 0.6623015999794006, 0.6623015999794006, 0.6690475940704346, 0.6742063760757446, 0.6674603223800659, 0.6650793552398682, 0.6734126806259155, 0.6317460536956787, 0.6682539582252502]


Для того чтобы улучшить модель можно 

1.   добавить скрытых слоев 
2.   уменьшить количество эпох так как там уже на плато вышли 



Может для того чтобы рекламу предлагать в зависимости от пола и.т.д

## Ответы:

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [551]:
current_datetime = datetime.now()
print(current_datetime)

2021-12-21 13:28:59.544244
